In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options 
import pandas as pd
import bs4
import requests
import re
import numpy as np

In [2]:
chrome_options = Options()  
chrome_options.add_argument("--headless") 
chrome_path=r"C:\Users\edward.chong\Desktop\chromedriver"
driver=webdriver.Chrome(chrome_path,chrome_options=chrome_options)

In [ ]:
driver.get('http://www.99chinaship.com/?go=setLanguage&lang=en')
records=[]
for i in range(1,101):
    driver.get('http://www.99chinaship.com/?go=shop&action=list&tid=88&page={}'.format(i))
    details_containers=driver.find_elements_by_css_selector('div.summary>h1>a')
    for details_container in details_containers:
        details_container.click()
        current_window = driver.current_window_handle
        new_window = [window for window in driver.window_handles if window != current_window][0]
        driver.switch_to.window(new_window)

        html=driver.page_source
        soup=bs4.BeautifulSoup(html,'lxml')
        sizes_list=[]
        colors_list=[]
        alt_images=[]
            
        if soup.find('h1',class_='ng-binding') != None:
            product_name=soup.find('h1',class_='ng-binding').text.strip()
            price_RM=soup.find('span',{ "id" : "gsItemPrice" }).text.strip()
            price_CNY=soup.find('span',class_='red price_cny ng-binding').text.strip()
            shipping_RM=soup.find('span',{ "id" : "post_fee" }).text.strip()
            shipping_CNY=soup.find('span',class_='red ng-binding').text.strip()
            image_url=soup.find('div',class_='jqzoom')
            image_srcs = [img['src'] for img in image_url.select('img[src]')]
            alt_image_container=soup.find('div',{ "id" : "spec-list" })
            alt_image_urls=alt_image_container.find_all('li')
            for alt_image_url in alt_image_urls:
                alt_images.append(alt_image_url('img')[0]['src'])
            sizes_container=soup.find_all('a', attrs={'class':'choose-sku' , 'id':re.compile('b_20509_')}) 
            for sizes in sizes_container:
                sizes_list.append(sizes.text)
            colors_container1=soup.find_all('a',attrs={'class':'choose-skuimg' ,'id':re.compile('b_1627207_')})
            for colors1 in colors_container1:
                colors_list.append(colors1.img['alt'] )
            colors_container2=soup.find_all('a',attrs={'class':'choose-sku' ,'id':re.compile('b_1627207_')})
            for colors2 in colors_container2:
                colors_list.append(colors2.text)
            colors_list=list(filter(None,colors_list))
            sizes_list=list(filter(None,sizes_list))
            page_num=i
            if soup.find('div',class_='col-md-12 goods-desc')!= None:
                descriptions_container=soup.find('div',class_='col-md-12 goods-desc')
                descriptions=descriptions_container.find_all('img')
                for description in descriptions:
                    if description.has_attr('data-original'):
                        description['src']=description['data-original']  
                descriptions=" ".join(str(x) for x in descriptions)
                descriptions=re.sub(r'width="(.*?)\"' ,'',descriptions)
            else:
                descriptions=""
                
            records.append((product_name,sizes_list,colors_list,price_RM,price_CNY,shipping_RM,shipping_CNY,image_srcs[0],alt_images,descriptions,page_num))
        
            driver.close()
            driver.switch_to.window(current_window)
        else:
            driver.close()
            driver.switch_to.window(current_window)

In [ ]:
df1=pd.DataFrame(records,columns=['Product Name','Sizes','Colors','Price(RM)','Price(CNY)','Shipping(RM)','Shipping(CNY)','Base Image','Alt Images','Description','Page #'])
df1['Product Name'].replace('', np.nan, inplace=True)
df1.dropna(subset=['Product Name'], inplace=True)

In [ ]:
df2=pd.DataFrame(columns=['Seller_id','model','sku','description:name:English','description:meta_title:English','description:description:English','quantity','stock_status','stock_status_name','image','productimages:image','shipping','price','tax_class','weight','weight_class','weight_class_name','category'])

In [ ]:
df2['description:name:English']=df1['Product Name']
df2['description:description:English']=df1['Description']
df2['model']=9940001+df2.index
df2['description:meta_title:English']=df2['description:name:English']
df2['sku']=df2['model']
df2['stock_status']=7
df2['stock_status_name']='In Stock'
df2['quantity']=10
df2['shipping']=1
df2['tax_class']=0
df2['weight']=0.5
df2['weight_class']=1
df2['weight_class_name']='Kilogram'
df2['category']="Fashion/Women's Fashion/Top/Blouse & Shirts"
df2['Seller_id']=35
df2['price']=df1['Price(RM)'].astype(float)*1.3
df2['price']=df2['price'].round(2)
df1['quantity']=df2['quantity']
df1['sku']=df2['sku']
df1['weight']=df2['weight']
df1['Adj_price']=df2['price']

In [ ]:
df1.Colors=df1.Colors.apply(lambda x: [s.replace(',','') for s in x])

In [ ]:
df2['image']=df1['Base Image'].fillna('')
df2['productimages:image']=df1['Alt Images'].fillna('')

def image_format(x):
    list1=re.split(r'/',x)
    
    if len(list1)>0:
        image_link='wkseller/35/'+ list1[-1][:-4]+'.jpg'
    else:
        image_link=''
    
    return image_link

df2['image']=df2['image'].apply(image_format)

In [ ]:
df1['color_filter']=df1.Colors.apply(lambda x:filter(None,['Color,'+s for s in x])).apply(lambda x:'|'.join(x))
df1['size_filter']=df1.Sizes.apply(lambda x:filter(None,['Size,'+s for s in x])).apply(lambda x: '|'.join(x))

In [ ]:
conditions = [
    (df1['color_filter'] != '') & (df1['size_filter'] != ''),
    (df1['color_filter'] != '') & (df1['size_filter'] == ''),
    (df1['color_filter'] == '') & (df1['size_filter'] != ''),
    (df1['color_filter'] == '') & (df1['size_filter'] == '')]
choices = ['Color + Size', 'Color', 'Size','']
df2['related:option'] = np.select(conditions, choices, default='')

In [ ]:
conditions1 = [
    (df2['related:option'] == 'Color + Size') | (df2['related:option'] == 'Color'),
    (df2['related:option'] == 'Size'),
    (df2['related:option'] == '')]
c1=df1[['color_filter','size_filter']].apply(lambda x: '|'.join(x),axis=1)
c2=df1[['color_filter','size_filter']].apply(lambda x: '|'.join(x)[1:],axis=1)
c3=''
choices1 = [c1,c2,c3]
df2['filter:filter'] = np.select(conditions1, choices1, default='')

In [ ]:
df2['description:description:English']=df2['description:description:English'].apply(lambda x: x[:32700])

In [ ]:
def related_value(x):
    list1=list(x)
    
    if (list1[0]==[] and list1[1]==[]):
        related_values=''
    elif (list1[0]!=[] and list1[1]==[]):
        list2=list1[0]
        list2=[','.join([s, str(list1[2]),str(list1[3])+'-'+str(list2.index(s)+1),str(list1[4]),str(list1[5])]) for s in list2]
        related_values='|'.join(list2)
    elif (list1[0]==[] and list1[1]!=[]):
        list2=list1[1]
        list2=[','.join([s, str(list1[2]),str(list1[3])+'-'+str(list2.index(s)+1),str(list1[4]),str(list1[5])]) for s in list2]
        related_values='|'.join(list2)
    else: 
        list2=[','.join([s1, s2]) for s1 in list1[0] for s2 in list1[1]]
        list2=[','.join([s, str(list1[2]),str(list1[3])+'-'+str(list2.index(s)+1),str(list1[4]),str(list1[5])]) for s in list2]
        related_values='|'.join(list2)
    
    return related_values

df2['related:value']=df1[['Colors','Sizes','quantity','sku','weight','Adj_price']].apply(related_value,axis=1)

In [ ]:
df2=pd.DataFrame({col:np.repeat(df2[col].values, df2['productimages:image'].str.len())for col in df2.columns.difference(['productimages:image'])}).assign(**{'productimages:image':np.concatenate(df2['productimages:image'].values)})[df2.columns.tolist()]

In [ ]:
df2['productimages:image']=df2['productimages:image'].apply(image_format)

In [ ]:
df2.to_csv('KnittedCardingan.csv',encoding='utf-8-sig',index=False)
driver.quit()

In [ ]:
df3=pd.DataFrame(columns=['Link','folder path','image name','seller_id'])
list_of_lists=df1['Alt Images'].tolist()
list_=df1['Base Image'].tolist()
flattened_list = [y for x in list_of_lists for y in x]
Combined_links=set(flattened_list+list_)
edited_list1=[]
edited_list2=[]
for link in Combined_links:
    if link[:1]!='h':
        link='https:'+link
    else:
        link=link
    edited_list1.append(link) 
df3['Link']=edited_list1
df3['seller_id']=35
df3['folder path']='/'
for link in Combined_links:
    seperated_link=link.split('/')
    image_name=seperated_link[-1][:-4]
    edited_list2.append(image_name)
df3['image name']=edited_list2
df3.to_csv('ImageUpload2.csv',encoding='utf-8-sig',index=False)

In [ ]:
for element in edited_list1:
    url = element
    filename = url.split('/')[-1][:-4]
    filepath='C:/Users/edward.chong/Desktop/ScrapImg/KnittedCardingan'
    r = requests.get(url, allow_redirects=True)
    open(filepath+'/'+filename+'.jpg', 'wb').write(r.content)

In [ ]:
df4=pd.DataFrame(columns=['option_value_id','option_id','sort-order','name(en-gb)'])
df4['name(en-gb)']=df2['filter:filter'].apply(lambda x: x.replace(',',':')).apply(lambda x: x.replace('|',',')).apply(lambda x: x.split(','))
df4=pd.DataFrame({col:np.repeat(df4[col].values, df4['name(en-gb)'].str.len())for col in df4.columns.difference(['name(en-gb)'])}).assign(**{'name(en-gb)':np.concatenate(df4['name(en-gb)'].values)})[df4.columns.tolist()]
df4['option_id']=df4['name(en-gb)'].apply(lambda x: x[:5])
df4=df4.drop_duplicates('name(en-gb)')
id_list=[]
for id in df4['option_id']:
    if id=='Color':
        id=2
    elif id=='Size:':
        id=5
    else:
        id=''
    id_list.append(id)
df4['name(en-gb)']=df4['name(en-gb)'].apply(lambda x: x.split(':')[-1])
df4['option_id']=id_list
df4['sort-order']=0
df4['option_value_id']=''
df4.to_csv('Variant2.csv',encoding='utf-8-sig',index=False)

In [ ]:
df1.to_csv('status2.csv',encoding='utf-8-sig',index=False)